# Ch 5 - Actor-Critic Models
### Deep Reinforcement Learning in Action

##### Listing 5.1

In [1]:
import multiprocessing as mp
import numpy as np
# jupyter에선 이런 방식의 multiprocessing이 작동하지 않음
# 따라서 worker.py 파일을 추가하여 원하는 함수를 따로 지정 
from worker import square
# def square(x): #A
#     print("x:",x)
#     return np.square(x)



In [2]:

num_cpu = mp.cpu_count()
x = np.arange(num_cpu * num_cpu) #B
print(x)
print(mp.cpu_count())


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143]
12


In [3]:

pool = mp.Pool(8) #C


In [4]:

squared = pool.map(square, [x[num_cpu*i:num_cpu*i+num_cpu] for i in range(num_cpu)])
print(squared)

[array([  0,   1,   4,   9,  16,  25,  36,  49,  64,  81, 100, 121]), array([144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529]), array([ 576,  625,  676,  729,  784,  841,  900,  961, 1024, 1089, 1156,
       1225]), array([1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116,
       2209]), array([2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364,
       3481]), array([3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900,
       5041]), array([5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724,
       6889]), array([7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836,
       9025]), array([ 9216,  9409,  9604,  9801, 10000, 10201, 10404, 10609, 10816,
       11025, 11236, 11449]), array([11664, 11881, 12100, 12321, 12544, 12769, 12996, 13225, 13456,
       13689, 13924, 14161]), array([14400, 14641, 14884, 15129, 15376, 15625, 15876, 16129, 16384,
       16641, 16900, 17161]), array([17424, 17689, 17956, 18225

##### Listing 5.2

In [5]:
from worker import square2
import sys

In [6]:

# def square2(i, x, queue):
#     print("In process {}".format(i,))
#     queue.put(np.square(x))

# 프로세스를 더 명시적으로 제어하고 공유 가능한 자료구조를 이용해서 프로세서들이 데이터 공유 가능 

if __name__ == '__main__':
    processes = [] #A  프로세스의 참조(포인터같은 느낌인가?)를 담을 리스트
    queue = mp.Queue() #B
    x = np.arange(64) #C
    for i in range(8): #D
        start_index = 8*i
        
        proc = mp.Process(target=square2,args=(i,x[start_index:start_index+8], queue)) 
        proc.start()
        processes.append(proc)
        
    for proc in processes: #E 모든 프로세스가 완료되길 기다림 
        proc.join()
        
    for proc in processes: #F 완료되었다면 모든 프로세스 종료 
        proc.terminate()

    results = []
    while not queue.empty(): #G queue에서 순서대로 결과를 담아옴 
        results.append(queue.get())

    print(results)

[array([ 64,  81, 100, 121, 144, 169, 196, 225]), array([ 0,  1,  4,  9, 16, 25, 36, 49]), array([1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521]), array([1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209]), array([2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025]), array([256, 289, 324, 361, 400, 441, 484, 529]), array([576, 625, 676, 729, 784, 841, 900, 961]), array([3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969])]


##### Listing 5.4

In [7]:
import torch
from torch import nn
from torch import optim
import numpy as np
from torch.nn import functional as F
import gym
import torch.multiprocessing as mp #A

class ActorCritic(nn.Module): #B
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(4,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,2)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.relu(self.l2(y))
        # l2를 통과한 결과로 actor와 critic을 모두 사용한다 
        # log_softmax(x) := log(softmax(x)) 와 같지만 따로 사용하는 것보다 안정성이 높음 
        actor = F.log_softmax(self.actor_lin1(y),dim=0) #C
        c = F.relu(self.l3(y.detach()))  # critic은 역전파되지 않음 
        # critic은 -1~1 사이의 값을 출력한다 
        # 이러한 값은 보상이 -1, +1인 cartpole과 잘 맞는다 
        critic = torch.tanh(self.critic_lin1(c)) #D
        return actor, critic #E

##### Listing 5.6

In [8]:
from worker import cworker

In [9]:
'''
def worker(t, worker_model, counter, params):
    worker_env = gym.make("CartPole-v1")
    worker_env.reset()
    # 하나의 모형을 모든 프로세스가 공유 
    worker_opt = optim.Adam(lr=1e-4,params=worker_model.parameters()) #A
    worker_opt.zero_grad()
    for i in range(params['epochs']):
        worker_opt.zero_grad()
        # episode를 진행하여 데이터를 수집하고 매개변수들을 갱신한다 
        values, logprobs, rewards = run_episode(worker_env,worker_model) #B 
        actor_loss,critic_loss,eplen = update_params(worker_opt,values,logprobs,rewards) #C
        counter.value = counter.value + 1 #D
'''

'\ndef worker(t, worker_model, counter, params):\n    worker_env = gym.make("CartPole-v1")\n    worker_env.reset()\n    # 하나의 모형을 모든 프로세스가 공유 \n    worker_opt = optim.Adam(lr=1e-4,params=worker_model.parameters()) #A\n    worker_opt.zero_grad()\n    for i in range(params[\'epochs\']):\n        worker_opt.zero_grad()\n        # episode를 진행하여 데이터를 수집하고 매개변수들을 갱신한다 \n        values, logprobs, rewards = run_episode(worker_env,worker_model) #B \n        actor_loss,critic_loss,eplen = update_params(worker_opt,values,logprobs,rewards) #C\n        counter.value = counter.value + 1 #D\n'

##### Listing 5.5

In [10]:
##### Listing 5.8
def update_params(worker_opt,values,logprobs,rewards,clc=0.1,gamma=0.95):
    # flip: 성분들을 모두 역순으로 정렬
    # view(-1): 평평하게 만듦, 1차원 배열이 아닌 텐서를 넘겨줄 수도 있기 때문 
    rewards = torch.Tensor(rewards).flip(dims=(0,)).view(-1) #A
    logprobs = torch.stack(logprobs).flip(dims=(0,)).view(-1)
    values = torch.stack(values).flip(dims=(0,)).view(-1)

    Returns = []
    ret_ = torch.Tensor([0])
    # 역순으로 return을 계산해서 Returns 배열에 저장
    for r in range(rewards.shape[0]): #B
        ret_ = rewards[r] + gamma * ret_
        Returns.append(ret_)
    Returns = torch.stack(Returns).view(-1)
    Returns = F.normalize(Returns,dim=0)
    # actor의 손실 계산, critic의 loss가 역전파되지 않도록 detach 사용 
    actor_loss = -1*logprobs * (Returns - values.detach()) #C
    # critic의 손실은 작을 수록 보상을 더 잘 예측하게 함 
    critic_loss = torch.pow(values - Returns,2) #D
    # critic의 loss 비율은 적절한 비율로 감소시킴 
    loss = actor_loss.sum() + clc*critic_loss.sum() #E
    loss.backward()
    worker_opt.step()
    return actor_loss, critic_loss, len(rewards)

##### Listing 5.7
def run_episode(worker_env, worker_model):
    state = torch.from_numpy(worker_env.env.state).float() #A
    values, logprobs, rewards = [],[],[] #B state value, log probability, reward
    done = False
    j=0
    while (done == False): #C
        j+=1
        policy, value = worker_model(state) #D
        values.append(value)
        logits = policy.view(-1)
        action_dist = torch.distributions.Categorical(logits=logits)
        action = action_dist.sample() #E  확률에 따라 sampling
        logprob_ = policy.view(-1)[action]
        logprobs.append(logprob_)
        # 뽑은 action에 따라 다음 step 진행 
        state_, _, done, info = worker_env.step(action.detach().numpy())
        state = torch.from_numpy(state_).float()
        if done: #F
            reward = -10
            worker_env.reset()
        else:
            reward = 1.0
        rewards.append(reward)
    return values, logprobs, rewards


# Listing 5.6
def cworker(t, worker_model, counter, params):
    # print("t is:",t, flush=True)
    sys.stdout = open(str(os.getpid()) + ".out", "w")
    info('function cworker')
    print ('hello')
    worker_env = gym.make("CartPole-v1")
    worker_env.reset()
    # 하나의 모형을 모든 프로세스가 공유 
    worker_opt = optim.Adam(lr=1e-4,params=worker_model.parameters()) #A
    worker_opt.zero_grad()
    for i in range(params['epochs']):
        worker_opt.zero_grad()
        # episode를 진행하여 데이터를 수집하고 매개변수들을 갱신한다 
        values, logprobs, rewards = run_episode(worker_env,worker_model) #B 
        actor_loss,critic_loss,eplen = update_params(worker_opt,values,logprobs,rewards) #C
        counter.value = counter.value + 1 #D

In [11]:
MasterNode = ActorCritic()
MasterNode.share_memory()  # 프로세스들이 model의 매개변수를 복사하는게 아니라 공유하게 함 
processes = []
params = {
    'epochs':1000,
    'n_workers':7,
}
# 내장 공유 객체를 전역 공유 카운터로 사용
counter = mp.Value('i',0)  # i는 정수라는 뜻 
if __name__ == '__main__': #adding this for process safety
    for i in range(params['n_workers']):
        p = mp.Process(target=cworker, args=(i,MasterNode,counter, params))
        p.start()
        processes.append(p)

    print(len(processes))
    for p in processes:
        p.join()

    for p in processes:
        p.terminate()
    # 전역 카운터의 값과 첫 프로세스의 종료코드 출력(문제 없으면 0)
    print(counter.value,processes[1].exitcode)

7
0 1


##### Test the trained agent

In [12]:
from IPython import display as ipythondisplay
from PIL import Image

In [14]:
env = gym.make("CartPole-v1", render_mode='rgb_array')
env.reset()
frames = []
screen = env.render()
images = [Image.fromarray(screen)]
for i in range(100):
    
    state_ = np.array(env.env.state)
    state = torch.from_numpy(state_).float()
    logits,value = MasterNode(state)
    action_dist = torch.distributions.Categorical(logits=logits)
    action = action_dist.sample()
    # print(env.step(action.detach().numpy()))
    state2, reward, done, _, info = env.step(action.detach().numpy())
    if done:
        print("Lost")
        env.reset()
    state_ = np.array(env.env.state)
    state = torch.from_numpy(state_).float()
    screen = env.render()
    images.append(Image.fromarray(screen))
    #frames.append(env.render())
    
env.close()
# images = render_episode(render_env, model, max_steps_per_episode)
image_file = 'cartpole-v1.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(image_file, save_all=True, append_images=images[1:], loop=0, duration=1)
# display_frames_as_gif(frames)


Lost
Lost
Lost
Lost


##### Listing 5.9

In [ ]:
def run_episode(worker_env, worker_model, N_steps=10):
    raw_state = np.array(worker_env.env.state)
    state = torch.from_numpy(raw_state).float()
    values, logprobs, rewards = [],[],[]
    done = False
    j=0
    G=torch.Tensor([0]) #A
    while (j < N_steps and done == False): #B
        j+=1
        policy, value = worker_model(state)
        values.append(value)
        logits = policy.view(-1)
        action_dist = torch.distributions.Categorical(logits=logits)
        action = action_dist.sample()
        logprob_ = policy.view(-1)[action]
        logprobs.append(logprob_)
        state_, _, done, info = worker_env.step(action.detach().numpy())
        state = torch.from_numpy(state_).float()
        if done:
            reward = -10
            worker_env.reset()
        else: #C
            reward = 1.0
            G = value.detach()
        rewards.append(reward)
    return values, logprobs, rewards, G

##### Listing 5.10

In [1]:
# bootstrapping의 유무에 따른 총 수익 변화 
#Simulated rewards for 3 steps
r1 = [1,1,-1]
r2 = [1,1,1]
R1,R2 = 0.0,0.0
#No bootstrapping
for i in range(len(r1)-1,0,-1): 
    R1 = r1[i] + 0.99*R1
for i in range(len(r2)-1,0,-1):
    R2 = r2[i] + 0.99*R2
print("No bootstrapping")
print(R1,R2)
#With bootstrapping
R1,R2 = 1.0,1.0
for i in range(len(r1)-1,0,-1):
    R1 = r1[i] + 0.99*R1
for i in range(len(r2)-1,0,-1):
    R2 = r2[i] + 0.r99*R2
print("With bootstrapping")
print(R1,R2)

No bootstrapping
0.010000000000000009 1.99
With bootstrapping
0.9901 2.9701
